In [1]:
import pandas as pd
from datetime import datetime
%load_ext autoreload
%autoreload 2
import utils
import warnings
warnings.filterwarnings("ignore")

In [2]:
df_reviews = pd.read_csv('reviews.csv')
df_games = pd.read_csv('games.csv')
df_items = pd.read_csv('items')
df_gastos_items = pd.read_csv('gastos_items.csv')
df_playtime_forever = pd.read_csv('playtime_forever.csv')
df_genre_ranking = pd.read_csv('genre_ranking.csv')
df_items_developer = pd.read_csv('items_developer.csv')

USER DATA

In [5]:
def userdata(user_id):
    
    # Filtra por el usuario de interés
    usuario = df_reviews[df_reviews['user_id'] == user_id]
    # Calcula la cantidad de dinero gastado para el usuario de interés
    cantidad_dinero = df_gastos_items[df_gastos_items['user_id']== user_id]['price'].iloc[0]
    # Busca el count_item para el usuario de interés    
    count_items = df_gastos_items[df_gastos_items['user_id']== user_id]['items_count'].iloc[0]
    
    # Calcula el total de recomendaciones realizadas por el usuario de interés
    total_recomendaciones = usuario['recommend'].sum()
    # Calcula el total de reviews realizada por todos los usuarios
    total_reviews = len(df_reviews['user_id'].unique())
    # Calcula el porcentaje de recomendaciones realizadas por el usuario de interés
    porcentaje_recomendaciones = (total_recomendaciones / total_reviews) * 100
    
    return {
        'cantidad_dinero': cantidad_dinero,
        'porcentaje_recomendacion': round(porcentaje_recomendaciones, 2),
        'total_items': count_items.astype(int)
    }

In [6]:
user_id = 'EchoXSilence'
userdata(user_id)

{'cantidad_dinero': 189.84, 'porcentaje_recomendacion': 0.0, 'total_items': 23}

COUNTREVIEWS

In [9]:
def countreviews(fecha_inicio, fecha_fin):
    
    # Filtra el dataframe entre las fechas de interés
    user_data_entre_fechas = df_reviews[(df_reviews['reviews_date'] >= fecha_inicio) & (df_reviews['reviews_date'] <= fecha_fin)]
    # Calcula la cantidad de usuarios que dieron reviews entre las fechas de interés
    total_usuarios = user_data_entre_fechas['user_id'].nunique()
    # Calcula el total de recomendaciones entre las fechas de interes (True + False)
    total_recomendacion = len(user_data_entre_fechas)
    # Calcula la cantidad de recomendaciones positivas que que hicieron entre las fechas de interés
    total_recomendaciones_True = user_data_entre_fechas['recommend'].sum()
    # Calcula el porcentaje de recomendación realizadas entre el total de usuarios
    porcentaje_recomendaciones = (total_recomendaciones_True / total_recomendacion) * 100
    
    return {
        'total_usuarios_reviews': total_usuarios,
        'porcentaje_recomendaciones': round(porcentaje_recomendaciones,2)
    }

In [22]:
countreviews('2011-04-01', '2012-04-30')

{'total_usuarios_reviews': 105, 'porcentaje_recomendaciones': 98.1}

In [ ]:
GENRE 

In [15]:
def genre(genero):
    # Busca el ranking para el género de interés
    rank = df_genre_ranking[df_genre_ranking['genres'] == genero]['ranking'].iloc[0]
    return {
        'rank': rank
    }

In [16]:
genero = 'Simulation'
genre(genero)

{'rank': 5}

USERFONGENRE

In [17]:
def userforgenre(genero):
    # Filtra el dataframe por el género de interés
    data_por_genero = df_playtime_forever[df_playtime_forever['genres'] == genero]
    # Agrupa el dataframe filtrado por usuario y suma la cantidad de horas
    top_users = data_por_genero.groupby(['user_url', 'user_id'])['playtime_horas'].sum().nlargest(5).reset_index()
    
    # Se hace un diccionario vacío para guardar los datos que se necesitan
    top_users_dict = {}
    for index, row in top_users.iterrows():
        # User info recorre cada fila del top 5 y lo guarda en el diccionario
        user_info = {
            'user_id': row['user_id'],
            'user_url': row['user_url']
        }
        top_users_dict[index + 1] = user_info
    
    return top_users_dict

In [18]:
genero = 'Action'
userforgenre(genero)

{1: {'user_id': 'Sp3ctre', 'user_url': 'http://steamcommunity.com/id/Sp3ctre'},
 2: {'user_id': 'shinomegami',
  'user_url': 'http://steamcommunity.com/id/shinomegami'},
 3: {'user_id': 'REBAS_AS_F-T',
  'user_url': 'http://steamcommunity.com/id/REBAS_AS_F-T'},
 4: {'user_id': 'Terminally-Chill',
  'user_url': 'http://steamcommunity.com/id/Terminally-Chill'},
 5: {'user_id': 'DownSyndromeKid',
  'user_url': 'http://steamcommunity.com/id/DownSyndromeKid'}}

DEVELOPER

In [19]:
def developer(desarrollador):
    # Filtra el dataframe por desarrollador de interés
    data_filtrada = df_items_developer[df_items_developer['developer'] == desarrollador]
    # Calcula la cantidad de items por año
    cantidad_por_año = data_filtrada.groupby('release_anio')['item_id'].count()
    # Calcula la cantidad de elementos gratis por año
    cantidad_gratis_por_año = data_filtrada[data_filtrada['price'] == 0.0].groupby('release_anio')['item_id'].count()
    # Calcula el porcentaje de elementos gratis por año
    porcentaje_gratis_por_año = (cantidad_gratis_por_año / cantidad_por_año * 100).fillna(0).astype(int)

    result_dict = {
        'cantidad_por_año': cantidad_por_año.to_dict(),
        'porcentaje_gratis_por_año': porcentaje_gratis_por_año.to_dict()
    }
    
    return result_dict

In [20]:
desarrollador = 'Mechanical Boss'
developer(desarrollador)

{'cantidad_por_año': {'2016': 2, '2017': 1},
 'porcentaje_gratis_por_año': {'2016': 50, '2017': 0}}